<a href="https://colab.research.google.com/github/k1mtaeye07/ai_programming/blob/main/w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#word2vec을 활용한 영화 추천시스템

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#konlpy, jdk설치
%%bash
apt-get update
sudo apt-get install openjdk-11-jdk
pip install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [917 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http:

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 14.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
#JAVA_HOME 환경변수 설정
%env JAVA_HOME "/usr/lib/jvm/java-11-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-11-openjdk-amd64"


In [7]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

file_path = './dataset/movie_vectors.csv'
movies_df = pd.read_csv(file_path)

# 벡터 데이터 리스트로 변환
movies_df['vector'] = movies_df['vector'].apply(lambda x: np.array(eval(x)))
display(movies_df[['title','nouns','vector']].head(2))

,title,nouns,vector
0,1917,"['제', '차', '세계대전', '한창', '독일군', '모든', '통신망', '...","[-0.0974993109703064, -0.1123259887099266, 0.0..."
1,21 브릿지: 테러 셧다운,"['뉴욕', '맨해튼', '중심', '경찰', '연쇄', '살해', '사건', '범...","[-0.020010648295283318, -0.0842740535736084, -..."


In [8]:
# Word2Vec 모델 로드
model_path = "./model/word2vec_movie.model"
model = Word2Vec.load(model_path)

In [23]:
# 특정키워드의 벡터값와 가장 유사한 벡터값들
keyword="아이언맨"

print(f"{keyword}의 벡터값 :  {model.wv[keyword]} \n")
print(f"{keyword}와 가장 유사한 벡터값들 \n")
li = model.wv.most_similar(keyword)
for x in li:
  print(x)

아이언맨의 벡터값 :  [-1.44350320e-01  3.79267707e-02  1.83328494e-01  2.32466042e-01
  2.96698064e-01 -2.44998336e-01 -1.22962333e-01  1.35348827e-01
  1.37487426e-01 -6.80540204e-02 -5.01119733e-01 -1.19910620e-01
 -3.62754941e-01 -2.76572138e-01  2.52460122e-01 -2.38268763e-01
 -5.50378077e-02  1.60128266e-01  2.14436859e-01 -3.07304949e-01
 -1.56702362e-02 -1.29044726e-02 -1.25702154e-02  5.59173264e-02
 -6.51577488e-02 -9.96534377e-02 -3.30154181e-01  7.29313269e-02
 -9.40835327e-02  3.18640500e-01  2.10543945e-01  2.66776234e-01
 -2.50065356e-01  1.50973022e-01 -3.75312269e-01  3.78653593e-02
  3.37523550e-01  2.56854087e-01 -1.65695116e-01  2.53768303e-02
 -1.64178893e-01  1.33454874e-01  1.19137958e-01  3.50128472e-01
  4.10470217e-02 -2.92547494e-01 -2.57205933e-01 -9.48061645e-02
 -9.78006423e-02  1.86018988e-01  1.70677289e-04  1.05651096e-01
 -4.97665554e-02 -4.41993833e-01 -2.55436212e-01  7.30374977e-02
  2.44007081e-01 -2.49234438e-01 -1.17216364e-01  4.83978540e-02
 -1.14384562

In [24]:
print(f"제일 맞지 않는것은? : {model.wv.doesnt_match(['아이언맨', '스타크', '플래시'])} \n")
print(f"negative 적용 하면? : {model.wv.most_similar(positive=['아이언맨','스타크'], negative=['플래시'], topn=3)} \n")

# 유사도 테스트
pairs = [
    ('아이언맨', '수트'),
    ('아이언맨', '어벤져스'),
    ('아이언맨', '스파이더맨'),
    ('아이언맨', '거지'),
    ('아이언맨', '백수')
]

for w1, w2 in pairs:
    print(f'{w1} =======> {w2}\t  {model.wv.similarity(w1, w2):.2f}')

제일 맞지 않는것은? : 플래시 

negative 적용 하면? : [('수트', 0.8713275194168091), ('스파이더맨', 0.8027282357215881), ('하이테크', 0.776225745677948)] 

아이언맨 =======> 수트	  0.91
아이언맨 =======> 어벤져스	  0.81
아이언맨 =======> 스파이더맨	  0.80
아이언맨 =======> 거지	  0.28
아이언맨 =======> 백수	  0.18


In [26]:
okt = Okt()

def extract_nouns(text):
    return okt.nouns(text)

# 벡터화 함수
def vectorize_summary(summary, model):
    words = summary
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# 추천 함수
def get_recommendations(input_text, model, data):
    input_nouns = extract_nouns(input_text)
    input_vector = vectorize_summary(input_nouns, model).reshape(1, -1)
    # 코사인 유사도 계산
    # cosine_similarity() : 입력 벡터와 데이터프레임의 모든 영화 벡터 간의 코사인 유사도를 계산
    similarities = cosine_similarity(input_vector, np.vstack(data['vector'].values))
    #각 영화에 대한 유사도 값을 저장
    data['similarity'] = similarities[0]

    # 유사도 높은 순 상위10개
    recommendations = data.sort_values(by='similarity', ascending=False).head(10)
    return recommendations[['title','similarity','page_url']]

input_text = "어벤져스 토니 스타크"
recommendations = get_recommendations(input_text, model, movies_df)

print("추천 영화 결과")
pd.set_option('display.max_colwidth', None)
display(recommendations)

추천 영화 결과


,title,similarity,page_url
7617,아이언맨 2,0.873430,https://movie.naver.com/movie/bi/mi/basic.nhn?code=49008
8482,아이언맨,0.851414,https://movie.naver.com/movie/bi/mi/basic.nhn?code=44885
2162,스파이더맨: 홈커밍,0.847563,https://movie.naver.com/movie/bi/mi/basic.nhn?code=135874
5815,아이언맨 3,0.822416,https://movie.naver.com/movie/bi/mi/basic.nhn?code=70254
3930,어벤져스: 에이지 오브 울트론,0.822389,https://movie.naver.com/movie/bi/mi/basic.nhn?code=98438
6595,어벤져스,0.801374,https://movie.naver.com/movie/bi/mi/basic.nhn?code=72363
8893,스파이더맨 3,0.793047,https://movie.naver.com/movie/bi/mi/basic.nhn?code=43157
5842,어벤져스 어셈블,0.791685,https://movie.naver.com/movie/bi/mi/basic.nhn?code=143881
1386,어벤져스: 인피니티 워,0.767937,https://movie.naver.com/movie/bi/mi/basic.nhn?code=136315
1313,스파이더맨: 뉴 유니버스,0.767503,https://movie.naver.com/movie/bi/mi/basic.nhn?code=171725
